#### [ 자연어 전처리 - 정제+토큰화 ]
- 정제 단계 (Cleaning Step)
- 토큰화 단계 (Tokenize Step)

[1] 모듈 로딩 및 데이터 준비<hr>

In [2]:
## 토큰화 관련 모듈
from nltk.tokenize import sent_tokenize, word_tokenize 
from nltk.tag import pos_tag
from nltk.corpus import stopwords
from string import punctuation
from nltk.stem import WordNetLemmatizer

In [3]:
## 데이터 파일
DATA_FILE   = '../data/test_data.txt'           ## 말뭉치 즉 코퍼스(Copous)     
STOP_WORD   = stopwords.words('english')        ## 불용어 즉, 분석에 의미없는 단어들
PUNCTUATION = punctuation                       ## 구두점


In [4]:
print(f'STOP_WORD   : {len(STOP_WORD)}개\n{STOP_WORD}')
print(f'PUNCTUATION : {len(PUNCTUATION)}개\n{PUNCTUATION}')

STOP_WORD   : 198개
['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'sh

[2] 텍스트 데이터 정제<hr>
- 영어 : 대소문자 일치
- 필요없는 기호, 문자 제거

In [5]:
## - 파일 데이터 읽어오기
with open(DATA_FILE, mode='r', encoding='utf-8') as f:
    fileData=f.read()

##- 확인
print(f'fileData==> {len(fileData)}개')

fileData==> 1851개


In [6]:
## 대소문자 일치 ==> 소문자
fileData=fileData.lower()

##- 확인
print(f'fileData==>\n {fileData[:30]}')

fileData==>
 what a merry-go-round is the e


[3] 문장 단위 토큰화 + 불용어 제거 + 원형 복원 <hr>

In [7]:
## [3-1] 문장 분리 후 문장 단위에서 토큰 분리
sentences = sent_tokenize(fileData)
sentences

['what a merry-go-round is the eighteenth collection by british fashion designer alexander mcqueen, made for the autumn/winter 2001 season of his fashion house alexander mcqueen.',
 "the collection drew on imagery of clowns and carnivals, inspired by mcqueen's feelings about childhood and his experiences in the fashion industry.",
 'the designs were influenced by military chic, cinema such as nosferatu (1922) and cabaret (1972), 1920s flapper fashion, and the french revolution.',
 'the palette comprised dark colours complemented with neutrals and muted greens.',
 'the show marked the first appearance of the skull motif that became a signature of the brand.',
 "the collection's runway show was staged on 21 february 2001 at the gatliff road warehouse in london, as part of london fashion week.",
 "it was mcqueen's final show in london; all his future collections were presented in paris.",
 'sixty-two looks were presented in the main runway show, with at least six more in the finale.',
 '[

In [ ]:
## --------------------------------------------------------------
## 함수기능 : 품사 기반 불필요 품사 제거 + 원형 복원 토큰 반환 
## 함수이름 : covertOriginal
## 매개변수 : pos_token_list - (단어, 품사)형태의 토큰 리스트
## 함수결과 : 불필요 품사 제거 + 원형 복원 토큰 리스트
## --------------------------------------------------------------
def covertOriginal(pos_token_list):
    ## 표제어 추출 인스턴스
    wnLemma = WordNetLemmatizer()

    ## 원형 복원 저장 
    result =[]
    print(pos_token_list)
    ## 형용사, 동사 경우 표제어 즉, 원형 복원
    for word, pos in pos_token_list:
            
        ## 형용사, 동사 경우 표제어 즉, 원형 복원
        if pos[:2] in ['JJ', 'VB']:
            result.append(wnLemma.lemmatize(word, 'a' if pos=='JJ' else 'v' ))
        elif pos not in ['DT', 'IN', 'CD', 'CC']:
            ## 토큰 합치기 => 불필요한 품사 제거한 토큰들 그대로 추가
            result.append(word)

    return result

In [ ]:
## [3-2] 문장 단위 전처리 된 문장 리스트 추출 

## - 문장별 토큰 리스트, 품사 토큰 리스트 저장 
sent_token_list, pos_token_list =[],[]

## - 문장별 토큰화 처리
for sent in sentences:
    ## 문장 단위로 구두점 제거, 토큰 분리
    for p in PUNCTUATION: 
        if p in sent: sent=sent.replace(p,'')
    
    ## 토큰 분리=> list 반환
    pos_token_list = pos_tag( word_tokenize(sent) )
    print(pos_token_list)

    ## 형용사 'JJ'=> 'a', 동사 'VB' => 'v'원형 복원 
    ## 불필요한 품사 제거한 토큰들 저장
    tokens = covertOriginal(pos_token_list)
    print(tokens)

    ## 토큰 리스트에서 불용어 제거
    words = [ w for w in tokens  if w not in STOP_WORD ]

    ## 문장 단위 토큰 저장 
    sent_token_list.append(words)
    
print(F'총 문장 수 : {len(sent_token_list)}개\n{sent_token_list}')

[('what', 'WP'), ('a', 'DT'), ('merrygoround', 'NN'), ('is', 'VBZ'), ('the', 'DT'), ('eighteenth', 'JJ'), ('collection', 'NN'), ('by', 'IN'), ('british', 'JJ'), ('fashion', 'NN'), ('designer', 'NN'), ('alexander', 'NN'), ('mcqueen', 'NN'), ('made', 'VBN'), ('for', 'IN'), ('the', 'DT'), ('autumnwinter', 'NN'), ('2001', 'CD'), ('season', 'NN'), ('of', 'IN'), ('his', 'PRP$'), ('fashion', 'NN'), ('house', 'NN'), ('alexander', 'NN'), ('mcqueen', 'NN')]
[('what', 'WP'), ('a', 'DT'), ('merrygoround', 'NN'), ('is', 'VBZ'), ('the', 'DT'), ('eighteenth', 'JJ'), ('collection', 'NN'), ('by', 'IN'), ('british', 'JJ'), ('fashion', 'NN'), ('designer', 'NN'), ('alexander', 'NN'), ('mcqueen', 'NN'), ('made', 'VBN'), ('for', 'IN'), ('the', 'DT'), ('autumnwinter', 'NN'), ('2001', 'CD'), ('season', 'NN'), ('of', 'IN'), ('his', 'PRP$'), ('fashion', 'NN'), ('house', 'NN'), ('alexander', 'NN'), ('mcqueen', 'NN')]
['what', 'merrygoround', 'be', 'eighteenth', 'collection', 'british', 'fashion', 'designer', 'al

[4] 단어 사전 생성 <hr>
- 토큰 -- 정수 인코딩
- 특수토큰 : 없는 토큰 UNK, 길이 맞춤용 토큰 PAD

(4-1) 중복 단어 제거

In [10]:

## - 문장의 토큰들 하나로 합치기
token_list = [ word  for sent in sent_token_list  for word in sent  ]

print(f'token_list => {token_list}')
print(f'token_list => {len(token_list)}개')

## - 토큰들 중복 제거
token_list = list(set(token_list))
print(f'token_list => {token_list}')
print(f'token_list => {len(token_list)}개')

token_list => ['merrygoround', 'eighteenth', 'collection', 'british', 'fashion', 'designer', 'alexander', 'mcqueen', 'make', 'autumnwinter', 'season', 'fashion', 'house', 'alexander', 'mcqueen', 'collection', 'draw', 'imagery', 'clowns', 'carnivals', 'inspire', 'mcqueens', 'feelings', 'childhood', 'experiences', 'fashion', 'industry', 'designs', 'influence', 'military', 'chic', 'cinema', 'nosferatu', 'cabaret', 'flapper', 'fashion', 'french', 'revolution', 'palette', 'comprise', 'dark', 'colours', 'complement', 'neutrals', 'mute', 'greens', 'show', 'mark', 'first', 'appearance', 'skull', 'motif', 'become', 'signature', 'brand', 'collections', 'runway', 'show', 'stag', 'february', 'gatliff', 'road', 'warehouse', 'london', 'part', 'london', 'fashion', 'week', 'mcqueens', 'final', 'show', 'london', 'future', 'collections', 'present', 'paris', 'sixtytwo', 'looks', 'present', 'main', 'runway', 'show', 'least', 'finale', 'show', 'stag', 'dark', 'room', 'carousel', 'centre', 'finale', 'lights

(4-2) 단어 사전 생성<hr>

In [11]:
### dict 타입으로 단어사전 생성
VOCAB_TO_IDX = {'<PAD>':0, '<UNK>':1 }
VOCAB_TO_IDX

{'<PAD>': 0, '<UNK>': 1}

In [12]:
## 토큰들에게 정수 숫자 부여
for idx, token in enumerate(token_list, 2):
    VOCAB_TO_IDX[token]=idx 

## idx => word로 변환
IDX_TO_TOKEN = {v:k for k,v in VOCAB_TO_IDX.items()}


In [13]:

print('VOCAB_TO_IDX=>\n', VOCAB_TO_IDX)
print('IDX_TO_TOKEN=>\n', IDX_TO_TOKEN)

VOCAB_TO_IDX=>
 {'<PAD>': 0, '<UNK>': 1, 'greens': 2, 'pose': 3, 'authors': 4, 'rear': 5, 'brand': 6, 'flapper': 7, 'neutrals': 8, 'influence': 9, 'stag': 10, 'critical': 11, 'merrygoround': 12, 'evil': 13, 'week': 14, 'french': 15, 'management': 16, 'feelings': 17, 'ensembles': 18, 'nosferatu': 19, 'centre': 20, 'present': 21, 'retrospective': 22, 'bricàbrac': 23, 'mute': 24, 'comprise': 25, 'least': 26, 'autumnwinter': 27, 'cabaret': 28, 'industry': 29, 'become': 30, 'signature': 31, 'discarded': 32, 'take': 33, 'complement': 34, 'positive': 35, 'room': 36, 'designs': 37, 'collections': 38, 'experiences': 39, 'part': 40, 'conglomerate': 41, 'palette': 42, 'mcqueens': 43, 'inspire': 44, 'relationship': 45, 'dress': 46, 'attract': 47, 'imagery': 48, 'british': 49, 'mark': 50, 'come': 51, 'road': 52, 'suffocating': 53, 'serve': 54, 'piles': 55, 'final': 56, 'gatliff': 57, 'academic': 58, 'designer': 59, 'cinema': 60, 'lvmh': 61, 'make': 62, 'springsummer': 63, 'luxury': 64, 'carousel': 

[5] 자연어 => 숫자 변환 <hr>

In [14]:
## 문장별 토큰 ==> 수치화 
SENT_NUM_LIST =[]
for sent_line in sent_token_list:
    setnums= [ VOCAB_TO_IDX[word] for word in sent_line]
    print('\n[문장]', sent_line)
    print('[수치]', setnums)
    SENT_NUM_LIST.append(setnums)



[문장] ['merrygoround', 'eighteenth', 'collection', 'british', 'fashion', 'designer', 'alexander', 'mcqueen', 'make', 'autumnwinter', 'season', 'fashion', 'house', 'alexander', 'mcqueen']
[수치] [12, 79, 80, 49, 119, 59, 78, 112, 62, 27, 99, 119, 104, 78, 112]

[문장] ['collection', 'draw', 'imagery', 'clowns', 'carnivals', 'inspire', 'mcqueens', 'feelings', 'childhood', 'experiences', 'fashion', 'industry']
[수치] [80, 111, 48, 115, 117, 44, 43, 17, 116, 39, 119, 29]

[문장] ['designs', 'influence', 'military', 'chic', 'cinema', 'nosferatu', 'cabaret', 'flapper', 'fashion', 'french', 'revolution']
[수치] [37, 9, 85, 72, 60, 19, 28, 7, 119, 15, 74]

[문장] ['palette', 'comprise', 'dark', 'colours', 'complement', 'neutrals', 'mute', 'greens']
[수치] [42, 25, 81, 100, 34, 8, 24, 2]

[문장] ['show', 'mark', 'first', 'appearance', 'skull', 'motif', 'become', 'signature', 'brand']
[수치] [70, 50, 71, 103, 88, 118, 30, 31, 6]

[문장] ['collections', 'runway', 'show', 'stag', 'february', 'gatliff', 'road', 'wareh

In [15]:
## 숫자 문장들 체크
for _ in SENT_NUM_LIST: print( _ )

[12, 79, 80, 49, 119, 59, 78, 112, 62, 27, 99, 119, 104, 78, 112]
[80, 111, 48, 115, 117, 44, 43, 17, 116, 39, 119, 29]
[37, 9, 85, 72, 60, 19, 28, 7, 119, 15, 74]
[42, 25, 81, 100, 34, 8, 24, 2]
[70, 50, 71, 103, 88, 118, 30, 31, 6]
[38, 96, 70, 10, 109, 57, 52, 113, 108, 40, 108, 119, 14]
[43, 56, 70, 108, 89, 38, 21, 66]
[82, 114, 21, 101, 96, 70, 26, 98]
[70, 10, 81, 36, 65, 20]
[98, 102, 51, 86, 55, 32, 116, 23, 5, 106, 97, 46, 13, 115, 107, 106, 3, 68]
[11, 75, 80, 90, 35, 47, 58, 95, 120, 91]
[43, 93, 70, 63, 12, 54, 121, 119, 29, 76, 94, 67, 53]
[69, 122, 87, 4, 33, 110, 15, 64, 92, 41, 61, 16, 112, 77, 45]
[18, 12, 73, 84, 112, 22, 78, 112, 105, 83]


[6] 패딩 : 모든 문장 길이 맞추기<hr>

In [16]:
## 문자별 단어 갯수 체크
length  = [ len(sent) for sent in SENT_NUM_LIST]

print(f'문장별 단어 개수 :  {length}')
print(f'가장 긴 문장    :  {max(length)}, 가장 짧은 문장 :{min(length)}')

문장별 단어 개수 :  [15, 12, 11, 8, 9, 13, 8, 8, 6, 18, 10, 13, 15, 10]
가장 긴 문장    :  18, 가장 짧은 문장 :6


In [17]:
## 문장별 길이 일치 => 가장 긴문장
MAX_LEN = max(length)
NUMS = len(SENT_NUM_LIST)
for idx in range(NUMS):
    sent_len = len(SENT_NUM_LIST[idx])
    if sent_len != MAX_LEN:
        for _ in range(MAX_LEN-sent_len):
            SENT_NUM_LIST[idx].append( VOCAB_TO_IDX['<PAD>'])

In [18]:
for _ in SENT_NUM_LIST:
    print(f'{len(_)}개 : {_}')

18개 : [12, 79, 80, 49, 119, 59, 78, 112, 62, 27, 99, 119, 104, 78, 112, 0, 0, 0]
18개 : [80, 111, 48, 115, 117, 44, 43, 17, 116, 39, 119, 29, 0, 0, 0, 0, 0, 0]
18개 : [37, 9, 85, 72, 60, 19, 28, 7, 119, 15, 74, 0, 0, 0, 0, 0, 0, 0]
18개 : [42, 25, 81, 100, 34, 8, 24, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
18개 : [70, 50, 71, 103, 88, 118, 30, 31, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0]
18개 : [38, 96, 70, 10, 109, 57, 52, 113, 108, 40, 108, 119, 14, 0, 0, 0, 0, 0]
18개 : [43, 56, 70, 108, 89, 38, 21, 66, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
18개 : [82, 114, 21, 101, 96, 70, 26, 98, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
18개 : [70, 10, 81, 36, 65, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
18개 : [98, 102, 51, 86, 55, 32, 116, 23, 5, 106, 97, 46, 13, 115, 107, 106, 3, 68]
18개 : [11, 75, 80, 90, 35, 47, 58, 95, 120, 91, 0, 0, 0, 0, 0, 0, 0, 0]
18개 : [43, 93, 70, 63, 12, 54, 121, 119, 29, 76, 94, 67, 53, 0, 0, 0, 0, 0]
18개 : [69, 122, 87, 4, 33, 110, 15, 64, 92, 41, 61, 16, 112, 77, 45, 0, 0, 0]
18개 : [18, 12, 73, 84, 112, 22, 78,

[7] 인코딩<hr>

In [19]:
import numpy as np 
from sklearn.preprocessing import OneHotEncoder 

dataNP = np.array(SENT_NUM_LIST)
dataNP.shape

(14, 18)

In [20]:
## 인코딩
ohEncoder = OneHotEncoder(sparse_output=False)
SENT_VEC=ohEncoder.fit_transform(dataNP)

In [21]:
SENT_VEC

array([[0., 1., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 1., ..., 0., 1., 0.]])